In [1]:
import re
import os
import sys
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import date

In [2]:
os.chdir("/Users/s0m08ao/Documents/data_science_anu")

In [3]:
###Reading the data
outcomes_df = pd.read_csv("outcomes.csv")
projects_df = pd.read_csv("projects.csv")
essays_df = pd.read_csv("essays.csv")

In [4]:
outcomes_df.shape

(619326, 12)

In [5]:
outcomes_df.iloc[103]

projectid                                         fff43985ceaabd4ae506f9dc88c53e0a
is_exciting                                                                      f
at_least_1_teacher_referred_donor                                              NaN
fully_funded                                                                     f
at_least_1_green_donation                                                      NaN
great_chat                                                                       f
three_or_more_non_teacher_referred_donors                                      NaN
one_non_teacher_referred_donor_giving_100_plus                                 NaN
donation_from_thoughtful_donor                                                 NaN
great_messages_proportion                                                      NaN
teacher_referred_count                                                         NaN
non_teacher_referred_count                                                     NaN
Name

In [6]:
outcomes_df.columns

Index(['projectid', 'is_exciting', 'at_least_1_teacher_referred_donor',
       'fully_funded', 'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_referred_donors',
       'one_non_teacher_referred_donor_giving_100_plus',
       'donation_from_thoughtful_donor', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count'],
      dtype='object')

In [7]:
projects_df.shape

(664098, 35)

In [8]:
projects_df.iloc[0]

projectid                                 316ed8fb3b81402ff6ac8f721bb31192
teacher_acctid                            42d43fa6f37314365d08692e08680973
schoolid                                  c0e6ce89b244764085691a1b8e28cb81
school_ncesid                                                   6.3627e+10
school_latitude                                                    36.5763
school_longitude                                                  -119.609
school_city                                                          Selma
school_state                                                            CA
school_zip                                                           93662
school_metro                                                           NaN
school_district                                 Selma Unified Sch District
school_county                                                       Fresno
school_charter                                                           f
school_magnet            

In [9]:
projects_df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_district', 'school_county',
       'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted'],
      dtype='object')

In [10]:
essays_df.shape

(664098, 6)

In [11]:
essays_df.head()

projectid                    teacher_acctid  \
0  ffffc4f85b60efc5b52347df489d0238  c24011b20fc161ed02248e85beb59a90   
1  ffffac55ee02a49d1abc87ba6fc61135  947066d0af47e0566f334566553dd6a6   
2  ffff97ed93720407d70a2787475932b0  462270f5d5c212162fcab11afa2623cb   
3  ffff7266778f71242675416e600b94e1  b9a8f14199e0d8109200ece179281f4f   
4  ffff418bb42fad24347527ad96100f81  e885fb002a1d0d39aaed9d21a7683549   

                                          title  \
0                                         iMath   
1                     Recording Rockin' Readers   
2  Kindergarten In Need of Important Materials!   
3                               Let's Find Out!   
4                        Whistle While We Work!   

                                   short_description  \
0  It is imperative that teachers bring technolog...   
1  Can you imagine having to translate everything...   
2  It takes a special person to donate to a group...   
3  My Kindergarten students come from a variety o...   
4  By using the cross curricular games requested,...   

                                      need_statement  \
0                       My students need four iPods.   
1                     My students need a camcorder.    
2  My students need 17 assorted classroom materia...   
3  My students need 25 copies of Scholastic's "Le...   
4  My students need grade level appropriate games...   

                                               essay  
0  I am a fourth year fifth grade math teacher. T...  
1  Can you imagine having to translate everything...  
2  Hi. I teach a wonderful group of 4-5 year old ...  
3  My Kindergarten students come from a variety o...  
4  All work and no play makes school a dull place...

In [12]:
essays_df.columns

Index(['projectid', 'teacher_acctid', 'title', 'short_description',
       'need_statement', 'essay'],
      dtype='object')

In [13]:
### Merging all three data frames on projectid column
data_frame = pd.read_csv("essays.csv")
data_frame = pd.merge(data_frame, outcomes_df, how = 'left', on = 'projectid')
data_frame = pd.merge(data_frame, projects_df, how = 'inner', on = 'projectid')

In [14]:
data_frame.shape

(664098, 51)

In [15]:
data_frame.columns

Index(['projectid', 'teacher_acctid_x', 'title', 'short_description',
       'need_statement', 'essay', 'is_exciting',
       'at_least_1_teacher_referred_donor', 'fully_funded',
       'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_referred_donors',
       'one_non_teacher_referred_donor_giving_100_plus',
       'donation_from_thoughtful_donor', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count',
       'teacher_acctid_y', 'schoolid', 'school_ncesid', 'school_latitude',
       'school_longitude', 'school_city', 'school_state', 'school_zip',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'school_year_round', 'school_nlns', 'school_kipp',
       'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_ar

### Converting all non-numeric data to numeric

In [16]:
data_frame['is_exciting'] = np.where(data_frame['is_exciting']=='t', 1, 0)

In [17]:
data_frame['at_least_1_teacher_referred_donor'] = np.where(data_frame['at_least_1_teacher_referred_donor']=='t', 1 , 0)

In [18]:
data_frame['great_chat'] = np.where(data_frame['great_chat']=='t', 1 , 0)

In [19]:
data_frame['fully_funded'] = np.where(data_frame['fully_funded']=='t', 1 , 0)

In [20]:
data_frame['at_least_1_green_donation'] = np.where(data_frame['at_least_1_green_donation']=='t', 1 , 0)

In [21]:
data_frame['donation_from_thoughtful_donor'] = np.where(data_frame['donation_from_thoughtful_donor']=='t', 1 , 0)

In [22]:
data_frame['three_or_more_non_teacher_referred_donors'] = np.where(data_frame['three_or_more_non_teacher_referred_donors']=='t', 1 , 0)

In [23]:
data_frame['one_non_teacher_referred_donor_giving_100_plus'] = np.where(data_frame['one_non_teacher_referred_donor_giving_100_plus']=='t', 1 , 0)

In [24]:
data_frame['teacher_referred_count'] = np.where(data_frame['teacher_referred_count']>=1, 1 , 0)

In [25]:
data_frame['non_teacher_referred_count'] = np.where(data_frame['non_teacher_referred_count']>=1, 1 , 0)

### Splitting data to train and test

In [26]:
data_frame["split"] = "train"
data_frame["split"][data_frame["date_posted"]>="2014-01-01"]= "test"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
data_frame.head()

projectid                  teacher_acctid_x  \
0  ffffc4f85b60efc5b52347df489d0238  c24011b20fc161ed02248e85beb59a90   
1  ffffac55ee02a49d1abc87ba6fc61135  947066d0af47e0566f334566553dd6a6   
2  ffff97ed93720407d70a2787475932b0  462270f5d5c212162fcab11afa2623cb   
3  ffff7266778f71242675416e600b94e1  b9a8f14199e0d8109200ece179281f4f   
4  ffff418bb42fad24347527ad96100f81  e885fb002a1d0d39aaed9d21a7683549   

                                          title  \
0                                         iMath   
1                     Recording Rockin' Readers   
2  Kindergarten In Need of Important Materials!   
3                               Let's Find Out!   
4                        Whistle While We Work!   

                                   short_description  \
0  It is imperative that teachers bring technolog...   
1  Can you imagine having to translate everything...   
2  It takes a special person to donate to a group...   
3  My Kindergarten students come from a variety o...   
4  By using the cross curricular games requested,...   

                                      need_statement  \
0                       My students need four iPods.   
1                     My students need a camcorder.    
2  My students need 17 assorted classroom materia...   
3  My students need 25 copies of Scholastic's "Le...   
4  My students need grade level appropriate games...   

                                               essay  is_exciting  \
0  I am a fourth year fifth grade math teacher. T...            0   
1  Can you imagine having to translate everything...            0   
2  Hi. I teach a wonderful group of 4-5 year old ...            0   
3  My Kindergarten students come from a variety o...            0   
4  All work and no play makes school a dull place...            0   

   at_least_1_teacher_referred_donor  fully_funded  at_least_1_green_donation  \
0                                  0             0                          0   
1                                  0             1                          1   
2                                  0             1                          1   
3                                  0             0                          0   
4                                  0             0                          1   

   ...    poverty_level    grade_level  fulfillment_labor_materials  \
0  ...     high poverty     Grades 3-5                          9.0   
1  ...  highest poverty  Grades PreK-2                         35.0   
2  ...  highest poverty  Grades PreK-2                         35.0   
3  ...     high poverty  Grades PreK-2                         30.0   
4  ...      low poverty     Grades 3-5                         17.0   

   total_price_excluding_optional_support  \
0                                  703.34   
1                                  207.43   
2                                  440.01   
3                                  186.58   
4                                  798.85   

   total_price_including_optional_support  students_reached  \
0                                  857.73              90.0   
1                                  244.04               8.0   
2                                  517.66              25.0   
3                                  219.51              24.0   
4                                  974.21              20.0   

   eligible_double_your_impact_match eligible_almost_home_match date_posted  \
0                                  f                          f  2009-10-11   
1                                  f                          f  2011-06-11   
2                                  t                          f  2010-09-11   
3                                  f                          f  2014-04-07   
4                                  f                          f  2009-04-03   

   split  
0  train  
1  train  
2  train  
3   test  
4  train  

[5 rows x 52 columns]

In [28]:
data_frame.columns

Index(['projectid', 'teacher_acctid_x', 'title', 'short_description',
       'need_statement', 'essay', 'is_exciting',
       'at_least_1_teacher_referred_donor', 'fully_funded',
       'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_referred_donors',
       'one_non_teacher_referred_donor_giving_100_plus',
       'donation_from_thoughtful_donor', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count',
       'teacher_acctid_y', 'schoolid', 'school_ncesid', 'school_latitude',
       'school_longitude', 'school_city', 'school_state', 'school_zip',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'school_year_round', 'school_nlns', 'school_kipp',
       'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_ar

In [29]:
text_vars=['title', 'short_description', 'need_statement', 'essay']

In [2]:
pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to /private/var/folders/06/snndyvkn1bd503wrd3jswr980000gp/T/pip-req-build-5pjb39hg
  Running command git clone -q https://github.com/hyperopt/hyperopt-sklearn.git /private/var/folders/06/snndyvkn1bd503wrd3jswr980000gp/T/pip-req-build-5pjb39hg
     |████████████████████████████████| 964 kB 443 kB/s eta 0:00:01
  Created wheel for hpsklearn: filename=hpsklearn-0.0.3-py3-none-any.whl size=26791 sha256=ae747f7aa6dde5cb9809b8172447f9ff7e5715a416a9dab5f7667d90892f0aff
  Stored in directory: /private/var/folders/06/snndyvkn1bd503wrd3jswr980000gp/T/pip-ephem-wheel-cache-wwtvbp5r/wheels/47/a5/46/9ca750026db9dfa5de4bf4836194554cb0e2e01a245588ea59
Successfully built hpsklearn
Note: you may need to restart the kernel to use updated packages.


## Classifier Model

In [40]:
### 'r' is needed to randomly split the train & test data
data_frame["r"] = np.random.uniform(0,1,size=len(data_frame))

### split_tr is used distinguish train and test data
split_tr = data_frame["split"]=="train" 

### y_train is used as 'y' to fit the model using train data 
y_train = data_frame['is_exciting'][split_tr].values

### 5 values between 0 and 1 for 5-fold cv
probs = np.arange(0,1. + 1./5, 1./5)

### for every interesting feature in essays_df 
for var in text_vars:
    
    ###feature new_var is used for every variable to see partial prediction of model from that variable
    new_var = var+"_pred_partial"
    data_frame[new_var] = 0.0
    
    ###using tfidf matrix for modelling essays data 
    vectorizer = TfidfVectorizer(min_df=2, use_idf=1, smooth_idf=1, sublinear_tf=1, ngram_range=(1,2), token_pattern=r"(?u)\b[A-Za-z0-9()\'\-?!\"%]+\b",
                                 norm='l2', max_features=1000000)
    ### fitting the tfidf vectorizer to the data
    vectorizer.fit(data_frame[var][(data_frame["split"]=="train") | (data_frame["split"]=="test")].values.astype('U'))
    
    ### tfidf_train matrix containing tfidf of all the words in train data
    tfidf_train = vectorizer.transform(data_frame[var][split_tr].values.astype('U'))
    
    print(tfidf_train.shape)
    ## tfidf_all matrix containing tfidf of all the words in both train & test data
    tfidf_all = vectorizer.transform(data_frame[var].values.astype('U')) 
    
    ### final model used in SGDClassifier
    model = SGDClassifier(penalty="l2",loss="log",fit_intercept=True, shuffle=True,n_iter_no_change=20, n_jobs=-1,alpha=0.000005)
    model.fit(tfidf_train, y_train)
    data_frame[var+"_pred"] = model.predict_proba(tfidf_all)[:,1]
    ### 5-fold cross validation
    for i in range(5):             
        split_test = (split_tr) & ((data_frame['r']>=probs[i]) & (data_frame['r']<probs[i+1]))            
        split_train = (split_tr) & ((data_frame['r']<probs[i]) | (data_frame['r']>=probs[i+1]))
        
        ### best model shown out of many trial models
        model_temp = SGDClassifier(penalty="l2",loss="log",fit_intercept=True, shuffle=True,n_iter_no_change=20, n_jobs=-1,alpha=0.000005)
        x_train = vectorizer.transform(data_frame[var][split_train].values.astype('U'))
        x_test = vectorizer.transform(data_frame[var][split_test].values.astype('U'))
        model_temp.fit(x_train, data_frame['is_exciting'][split_train].values)                       
        pred_test = model_temp.predict_proba(x_test)[:,1]
        pred_train = model_temp.predict_proba(x_train)[:,1] 
        
        ### current variable's prediction is stored in the data_frame for next iteration 
        data_frame[new_var][split_test] = pred_test
        print('CV: ' + str(i+1))
        
        ### mean auc of both train and test auc score
        roc=(metrics.roc_auc_score(data_frame['is_exciting'][split_train],pred_train)+metrics.roc_auc_score(data_frame['is_exciting'][split_test],pred_test))/2
        print('AUC: ' + str(roc))

(619326, 189722)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CV: 1
AUC: 0.6699131407062422


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CV: 2
AUC: 0.6690594856006946


...

CV: 3
AUC: 0.788144623673544


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CV: 4
AUC: 0.7873126551286379


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CV: 5
AUC: 0.7859903906231459


In [41]:
pred_train_list = list(zip(pred_train, data_frame['projectid'][split_train]))

In [42]:
df_pred_train = pd.DataFrame(pred_train_list, columns = ['is_exciting', 'projectid'])

In [44]:
### dataframe for predictions.csv file
pred_test_list = list(zip(pred_test, data_frame['projectid'][split_test]))
df_pred_test = pd.DataFrame(pred_test_list, columns = ['is_exciting', 'projectid'])

In [46]:
df_pred = pd.concat([df_pred_train, df_pred_test])

In [47]:
df_pred['is_exciting'] = df_pred['is_exciting'].round(2)

In [48]:
df_pred.to_csv('predictions.csv', index = False)

In [49]:
### dataframe for fold_auc.csv file
data_fold_auc = [['1', 0.7862385072953801], ['2', 0.7864685331639762], ['3', 0.788144623673544],
                 ['4', 0.7873126551286379], ['5', 0.7859903906231459]]
df_fold_auc = pd.DataFrame(data_fold_auc, columns = ['Fold', 'AUC']) 

In [51]:
df_fold_auc.to_csv('fold_auc.csv', index = False)